In [5]:
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [6]:
train_images = np.random.randint(256, size=(20,32,32,3))
train_labels = np.random.randint(2, size=(20,1))
print(train_images.shape, train_labels.shape)

(20, 32, 32, 3) (20, 1)


In [7]:
class TensorData(Dataset):
    # 외부에 있는 데이터를 가져오기 위해 외부에서 데이터가 들어올 수 있도록, x_data, y_data 변수를 지정
    def __init__(self, x_data, y_data):
        #들어온 x는 tensor형태로 변환
        self.x_data = torch.FloatTensor(x_data)
        # tensor data의 형태는 (배치사이즈, 채널사이즈, 이미지 너비, 높이)의 형태임
        # 따라서 들어온 데이터의 형식을 permute함수를 활용하여 바꾸어주어야함.
        self.x_data = self.x_data.permute(0,3,1,2)  # 인덱스 번호로 바꾸어주는 것 # 이미지 개수, 채널 수, 이미지 너비, 높이
        self.y_data = torch.LongTensor(y_data) # float tensor / long tensor 로 숫자 속성을 정해줄 수 있음
        self.len = self.y_data.shape[0]

    # x,y를 튜플형태로 바깥으로 내보내기
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [8]:
# 인스터스(데이터) 생성
train_data = TensorData(train_images, train_labels)

# 만들어진 데이터가 배치형태로 만들어줘야하니까 DataLoader에다가 넣어줌
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)

In [29]:
train_data[0]

(tensor([[[189.,   4., 248.,  ..., 169.,  88., 238.],
          [ 60., 138., 171.,  ..., 170., 129., 200.],
          [ 26.,  64.,  47.,  ..., 207.,  20., 225.],
          ...,
          [167., 243., 240.,  ..., 177., 247.,  23.],
          [161.,   4., 151.,  ..., 217.,  75.,  81.],
          [186.,  55.,   2.,  ...,  82.,  56.,   1.]],
 
         [[117., 218., 145.,  ...,  70.,  57., 104.],
          [ 57.,  41., 187.,  ...,  58., 181., 204.],
          [188.,  75., 166.,  ..., 108., 215.,  43.],
          ...,
          [159., 146., 145.,  ..., 203., 149.,  24.],
          [244., 166., 177.,  ..., 251., 119., 102.],
          [222., 214., 166.,  ..., 186., 114.,  80.]],
 
         [[  7., 233.,  91.,  ..., 231., 154., 130.],
          [213.,   9., 152.,  ..., 159., 247., 187.],
          [ 44., 201.,  10.,  ...,  67., 240.,  95.],
          ...,
          [ 57., 167., 136.,  ..., 128., 170., 238.],
          [ 98., 175., 241.,  ..., 165., 139., 182.],
          [ 90., 177., 202.,  .

In [31]:
train_loader

In [9]:
from PIL import Image

def pil_loader(path):
    with open(path, 'rb') as f: # Read binary: PIL image는 binary로 읽어야
        with Image.open(f) as img: # Open image
            #(H, W, 3)
            return img.convert('RGB')

In [10]:
img=pil_loader('0000000000.png')

In [13]:
type(img)

PIL.Image.Image

In [20]:
from torchvision import transforms
img_tensor=transforms.Resize((192, 640), interpolation=Image.ANTIALIAS)(img)
img_tensor=transforms.ToTensor()(img_tensor)

C:\Users\wongyun\AppData\Local\Temp\ipykernel_18800\1241825421.py:2: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img_tensor=transforms.Resize((192, 640), interpolation=Image.ANTIALIAS)(img)
c:\Users\wongyun\miniconda3\envs\torchstudy\lib\site-packages\torchvision\transforms\transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [22]:
img_tensor.shape

torch.Size([3, 192, 640])

In [1]:
from pathlib import Path
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from superglue_networks import *

In [2]:
def match_points(
        input_pairs, resize_float=True, max_length=-1, 
        superglue='outdoor', max_keypoints=1024, keypoint_threshold=0.005,
        nms_radius=4, sinkhorn_iterations=20, match_threshold=0.2, 
        resize=[640, 480], shuffle=True):

    torch.set_grad_enabled(False)
    if len(resize) == 2 and resize[1] == -1:
        resize = resize[0:1]
    if len(resize) == 2:
        print('Will resize to {}x{} (WxH)'.format(
            resize[0], resize[1]))
    elif len(resize) == 1 and resize[0] > 0:
        print('Will resize max dimension to {}'.format(resize[0]))
    elif len(resize) == 1:
        print('Will not resize images')
    else:
        raise ValueError('Cannot specify more than two integers for --resize')

    scene0, scene1 =input_pairs

    # Load the SuperPoint and SuperGlue models.
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    #print('Running inference on device \"{}\"'.format(device))
    config = {
        'superpoint': {
            'nms_radius': nms_radius,
            'keypoint_threshold': keypoint_threshold,
            'max_keypoints': max_keypoints
        },
        'superglue': {
            'weights': superglue,
            'sinkhorn_iterations': sinkhorn_iterations,
            'match_threshold': match_threshold,
        }
    }
    matching = Matching(config).eval().to(device)

    timer = AverageTimer(newline=True)
    match_index = []
    
    pred = matching({'image0': input_pairs[0], 'image1': input_pairs[1]})
    pred = {k: v[0].cpu().numpy() for k, v in pred.items()}
    kpts0, kpts1 = pred['keypoints0'], pred['keypoints1']
    matches, conf = pred['matches0'], pred['matching_scores0']
    timer.update('matcher')
    # Write the matches to disk.
    out_matches = {'keypoints0': kpts0, 'keypoints1': kpts1,
                   'matches': matches, 'match_confidence': conf}
    match_index.append(out_matches)
    #for i, pair in enumerate(pairs): 안에
    # return here
    return match_index

In [1]:
import cv2

In [3]:
img1=cv2.imread("0000000000.png", 0)
img1.shape

(375, 1242)